# **Final Robustness Test**

- This will be with the annotated test data


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For preprocessing
import tensorflow as tf

# For modeling
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import load_model

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
import random

In [3]:
pkl_path_2D = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated Test/test_features_3s_all_2D.pkl'

# Load the pickle file
with open(pkl_path_2D, 'rb') as file:
    test_2D = pickle.load(file)
del file

In [4]:
test_f_2D = test_2D['test'].copy()
test_labels = test_f_2D['label'].copy()

temp = test_f_2D.copy()
del temp['label']
test_features_2D = temp

## **Shuffling Data**

In [5]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1827)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [6]:
test_y_2D, test_X_2D = shuffle_data(test_labels, test_features_2D)

## **CNN Model**

In [7]:
def tile_and_crop(feature, target_size):
    tiled = np.tile(feature, (1, target_size // feature.shape[1] + 1, 1))
    return tiled[:, :target_size, :]

In [8]:
test_results = {}

# **Deep Learning Models**

In [9]:
# 2D
Res_melmfcccqt = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/ResNet50_melspectrogram_cqt_mfcc_balanced.keras')
VGG_melmfcccqt = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/VGG16_melspectrogram_cqt_mfcc_balanced.keras')

Res_melmfccchroma = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/ResNet50_melspectrogram_chroma_mfcc_balanced.keras')
VGG_melmfccchroma = load_model('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/Transfer Learning/VGG16_melspectrogram_chroma_mfcc_balanced.keras')

# **Preparing Test Data**

## 2D Testing Stuff

In [10]:
target_size = 128
mfcc_tiled = tile_and_crop(test_X_2D['mfcc'], target_size)
chroma_tiled = tile_and_crop(test_X_2D['chroma'], target_size)
cqt_tiled = tile_and_crop(test_X_2D['cqt'], target_size)

testing_2D_melmfccchroma = np.stack((test_X_2D['melspectrogram'], mfcc_tiled, chroma_tiled), axis=-1)
testing_2D_melmfcccqt = np.stack((test_X_2D['melspectrogram'], mfcc_tiled, cqt_tiled), axis=-1)

# **Evaluation**

Mel Spectrogram/MFCC/CQT Models

In [11]:
accuracy = Res_melmfcccqt.evaluate(testing_2D_melmfcccqt, test_y_2D)[-1]

test_results['Res_CQT'] = accuracy

print(f"Test Accuracy: {accuracy}")

76/76 ━━━━━━━━━━━━━━━━━━━━ 15s 93ms/step - accuracy: 0.7494 - loss: 1.9455
Test Accuracy: 0.7666941285133362


In [12]:
accuracy = VGG_melmfcccqt.evaluate(testing_2D_melmfcccqt, test_y_2D)[-1]

test_results['VGG_CQT'] = accuracy

print(f"Test Accuracy: {accuracy}")

76/76 ━━━━━━━━━━━━━━━━━━━━ 26s 198ms/step - accuracy: 0.7024 - loss: 2.1035
Test Accuracy: 0.7192910313606262


# **Evaluation**

Mel Spectrogram/MFCC/Chroma Models

In [13]:
accuracy = Res_melmfccchroma.evaluate(testing_2D_melmfccchroma, test_y_2D)[-1]

test_results['Res_Chroma'] = accuracy

print(f"Test Accuracy: {accuracy}")

76/76 ━━━━━━━━━━━━━━━━━━━━ 10s 72ms/step - accuracy: 0.8395 - loss: 1.0297
Test Accuracy: 0.8388293385505676


In [14]:
accuracy = VGG_melmfccchroma.evaluate(testing_2D_melmfccchroma, test_y_2D)[-1]

test_results['VGG_Chroma'] = accuracy

print(f"Test Accuracy: {accuracy}")

76/76 ━━━━━━━━━━━━━━━━━━━━ 7s 82ms/step - accuracy: 0.8698 - loss: 0.7568
Test Accuracy: 0.865210235118866


## Review all the results

In [15]:
test_results_df = pd.DataFrame(list(test_results.items()), columns=['Models', 'Testing_Accuracy']).round(3)

test_results_df = test_results_df.sort_values('Models')
test_results_df

,Models,Testing_Accuracy
0,Res_CQT,0.767
2,Res_Chroma,0.839
1,VGG_CQT,0.719
3,VGG_Chroma,0.865
